<a href="https://colab.research.google.com/github/2018007956/HYU/blob/main/DeepLearning/13_DQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DQN architecture: CNN + FC  
Environment setting: Gym

코드 수정 필요!!

In [ ]:
class DON (nn. Module)
def __init__(self, h. w, outputs) :
super (DON, self) .__init__(
self.conv1 = nn .Conv2d(3. 16, kernel _size=5, stride=2)
self.bn1 = nn. BatchNorm2d(16)
self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
self.bn2 = nn. BatchNorm2d(32)
self.conv3 = nn. Conv2d(32, 32. kernel_size=5, stride=2)
self.bn3 = nn. BatchNorm2d (32)
# Number of Linear input connections depends on output of conv2d layers
# and therefore the input image size, so compute it.
def conv2d_size_out (size, kernel_size = 5, stride = 2) :
return (size - (kernel_size - 1) - 1) // stride + 1
convw = conv2d_size_out (conv2d_size_out (conv2d_size_out (w) ))
convh = conv2d_size_out (conv2d_size_out (conv2d_size_out (h)))
linear_ input_ size = convw * convh * 32
self. head = nn.Linear (linear_input _size, outputs)
# Called with either one element to determine next action, or a batch
# during optimization. Returns tensor (I[leftOexp, rightOexp]..
.]) .
def forward(self, x) :
× = x. to (device)
× = F.relu(self.bn1 (self.conv1 (x)))
× = F.relu(self.bn2(self .conv2(x)))
× = F.relu(self.bn3(self .conv3 (x)))
return self.head(x. view(x. size (0) .
-1))

In [ ]:
resize = T.Compose ([T. ToP IL| mage ().
T.Resize (40. interpolation=|mage CUBIC).
T.ToTensor (1)
def get_cart_location (screen _width) :
world width = env.× threshold * 2
scale = screen _width / world width
return int (env.state[0] * scale + screen _width / 2.0) # MIDDLE OF CART
def get_screen:
# Returned screen requested by gym is 400×600×3, but is sometimes larger # such as 800×1200×3. Transpose it into torch order (CHW).
screen = env. render (mode= 'rgb _array'). transpose ((2, 0, 1))
# Cart is in the lower half, so strip off the top and bottom of the screen
-. screen _height, screen_width = screen. shape
screen = screen[:, int(screen_height*0.4) :int (screen_height * 0.8)]
view_width = int (screen _width * 0.6)
cart_location = get _cart_location (screen_width)
if cart_ location < view width // 2:
slice_range = slice(view width)
elif cart_location > (screen _width - view_width / / 2) :
slice_range = slice(-view_width, None)
else:
slice_range = slice(cart_location - view_width // 2.
cart_location + view_width // 2)
# Strip off the edges, so that we have a square image centered on a cart
screen = screen [:.
. slice_range]
# Convert to float, rescale, convert to torch tensor
# (this doesn't require a copy)
screen = np. ascontiguousarray(screen, dtype=np. float32) / 255
screen = torch. from_numpy (screen)
# Resize, and add a batch dimension (BCHW)
return resize(screen) .unsqueeze(0)

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS DECAY = 200
TARGET_UPDATE = 10
# Get screen size so that we can initialize layers correctly based on shape
# returned from Al gym. Typical dimensions at this point are close to 3x40×90
# which is the result of a clamped and down-scaled render buffer in get_screen ()
init_screen = get_screen ()
screen_height, screen_width = init_screen. shape
# Get number of actions from gym action space
n_actions = env .action_space.n
policy_net = DON(screen_height, screen_width, _actions). to(device)
target_net = DON(screen _height, screen_width, _actions). to (device)
target_net. load_state_dict(policy_net.state_dict))
target_net.eval ()
optimizer = optim.RMSprop(policy_net.parameters ())
memory = ReplayMemory (10000)
steps_done = 0

In [ ]:
def select_action (state):
global steps_done
sample = random. random()
eps_threshold = EPS_END + (EPS_START - EPS_END) * H
math.exp(-1
* steps done / EPS DECAY)
steps_done += 1
if sample > eps_threshold:
with torch.no_grad():
# t. max (1) will return largest column value of each row.
# second column on max result is index of where max element was
# found, so we pick action with the larger expected reward.
return policy_net (state) .max (1) [1]. view(1, 1)
else:
return torch. tensor ([[random.randrange (n_actions)]]. device=device, dtype=torch. long)

In [ ]:
def optimize model ():
if len (memory) < BATCH_ SIZE:
return
transitions = memory. sample (BATCH_ SIZE)
# Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
# detailed explanation). This converts batch-array of Transitions
# to Transition of batch-arrays.
batch = Transition(*zip(*transitions))
# Compute a mask of non-final states and concatenate the batch elements
# (a final state would've been the one after which simulation ended)
non_final_mask = torch. tensor (tuple(map(lambda s: s is not None.
batch.next_state)). device=device. type=torch.boo
non _final_next_states = torch.cat(ls for s in batch.next_state
if s is not Nonel)
state_batch = torch. cat (batch. state)
action_batch = torch. cat (batch.action)
reward_batch = torch.cat (batch.reward)
# Compute Q(s_t, a)
the model computes Q(s_t). then we select the
# columns of actions taken. These are the actions which would've been taken
# for each batch state according to policy_net
state_action_values = policy_net (state_batch) .gather (1. action_batch)
# Compute V(s_(t+1]) for all next states.
# Expected values of actions for non_ final _next_states are computed based
# on the "older" target_net: selecting their best reward with max (1) [0]
# This is merged based on the mask, such that we Il have either the expected
# state value or 0 in case the state was final.
next_state_values = torch.zeros (BATCH_SIZE, device=device)
next_state_values [non_final _mask] = target_net (non_final _next_states) . max (1) |[0] .detach ()
# Compute the expected Q values
expected_state_action_values = (next_state_values * GAMMA)
+ reward batch
# Compute Huber loss
criterion = nn. SmoothL1Loss ()
loss = criterion (state_action_values, expected state.
values .unsqueeze(1))
# Optimize the model
optimizer.zero grad()
loss .backward()
for param in policy_net parameters) :
param.grad. data.clamp_(-1, 1) optimizer .step()

In [ ]:
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400,900))
display.start()

In [ ]:
### DQN Training
num_episodes = 50
for i_episode in range (num_episodes) :
# Initialize the environment and state
env.reset ()
last_screen = get_screen ()
current_screen = get_screen (
state = current_screen - last_screen
for + in count:
# Select and perform an action
action = select action(state)
reward, done,_ = env.step(action.item())
reward = torch. tensor ([reward], device=device)
# Observe new state
last_screen = current_screen
current_screen = get_screen ()
if not done:
next_state = current_screen - last_screen
else:
next_state = None
# Store the transition in memory
memory.push (state, action, next_state, reward)
# Move to the next state
state = next state
# Perform one step of the optimization (on the policy network)
optimize_model ()
if done:
episode_durations. append (t + 1)
plot_durations ( break
# Update the target network, copying all weights and biases in DON
if i_episode % TARGET_UPDATE == 0:
target_net.load_state_dict(policy_net.state_dict())

In [ ]:
env = gym. make ( 'CartPole-v0')
env = gym.wrappers. Monitor (env, f"videos") # record videos
env = gym. wrappers. RecordEpisodeStatistics(env) # record stats such as returns
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
  from Python import display
pit.ion ()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wand. init(project="week14_dqn", config=config. monitor_gym=True) 
wandb.run.name = "dqn _cartpole"